In [1]:
from mylib.statistic_test import *

code_id = '0872 - Show aPVC with rPVC'
loc = os.path.join(figpath, 'Dsp', code_id)
mkdir(loc)
pass

d:\Software\Anaconda3\envs\maze\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


        D:\Data\FinalResults\Dsp\0872 - Show aPVC with rPVC is already existed!


In [ ]:
if os.path.exists(os.path.join(figdata, f'{code_id} [aPVC].pkl')) == False:
    aPVC = {
        "Pos": [],
        "aPVC": [],
        "Route": [],
        "MiceID": [],
        "Session": []
    }
    for mouse in [10212, 10224, 10227, 10232]:
        file_idx = np.where(f2['MiceID'] == mouse)[0]
        for s in tqdm(range(7)):
            with open(f2['Trace File'][file_idx[s]], 'rb') as handle:
                trace = pickle.load(handle)
            
            pos, apvc, rts = SegmentedCorrelationAcrossRoutes_DSP_Interface(trace, ['Position', 'aPVC', 'Routes'])
            aPVC['Pos'].append(pos)
            aPVC['aPVC'].append(apvc)
            aPVC['Route'].append(rts)
            aPVC['MiceID'].append(np.repeat(mouse, pos.shape[0]))
            aPVC['Session'].append(np.repeat(s, pos.shape[0]))
    
    for key in aPVC.keys():
        aPVC[key] = np.concatenate(aPVC[key])
    
    with open(os.path.join(figdata, f'{code_id} [aPVC].pkl'), 'wb') as handle:
        pickle.dump(aPVC, handle)
    
    aPVCD = pd.DataFrame(aPVC)
    aPVCD.to_excel(os.path.join(figdata, f'{code_id} [aPVC].xlsx'), index=False)
    print(len(aPVCD))
else:
    with open(os.path.join(figdata, f'{code_id} [aPVC].pkl'), 'rb') as handle:
        aPVC = pickle.load(handle)
        
if os.path.exists(os.path.join(figdata, f'{code_id}  [rPVC].pkl')) == False:
    rPVC = {
        "Pos": [],
        "rPVC": [],
        "Route": [],
        "MiceID": [],
        "Type": [],
        "Session": []
    }
    for mouse in [10212, 10224, 10227, 10232]:
        file_idx = np.where(f2['MiceID'] == mouse)[0]
        for s in tqdm(range(7)):
            with open(f2['Trace File'][file_idx[s]], 'rb') as handle:
                trace = pickle.load(handle)
            
            pos, rpvc, rts, isctrl = SegmentedCorrelationAcrossRoutes_Egocentric_DSP_Interface(trace, ['Position', 'rPVC', 'Routes', 'Type'])
            rPVC['Pos'].append(pos)
            rPVC['rPVC'].append(rpvc)
            rPVC['Route'].append(rts)
            rPVC['MiceID'].append(np.repeat(mouse, pos.shape[0]))
            rPVC['Type'].append(isctrl)
            rPVC['Session'].append(np.repeat(s, pos.shape[0]))
    
    for key in rPVC.keys():
        rPVC[key] = np.concatenate(rPVC[key])
    
    with open(os.path.join(figdata, f'{code_id} [rPVC].pkl'), 'wb') as handle:
        pickle.dump(rPVC, handle)
    
    rPVCD = pd.DataFrame(rPVC)
    rPVCD.to_excel(os.path.join(figdata, f'{code_id} [rPVC].xlsx'), index=False)
    print(len(rPVCD))
else:
    with open(os.path.join(figdata, f'{code_id} [rPVC].pkl'), 'rb') as handle:
        rPVC = pickle.load(handle)
        
def set_pos_offset(rt):
    for i, b in enumerate(CP_DSPs[1][0]):
        if b in CP_DSPs[1][rt]:
            return i
    return 111

ctrl = sns.color_palette("Greens", 4)
real = sns.color_palette("Purples", 4)
for rt in range(1, 7):
    pos_offset = set_pos_offset(rt)
    apvc_sub = SubDict(aPVC, aPVC.keys(), np.where((np.isin(aPVC['Route'], [0, rt])))[0])
    apvc_sub['Pos'] -= pos_offset
    rPVC_sub = SubDict(rPVC, rPVC.keys(), np.where((rPVC['Route']==rt)&(rPVC['Session'] == 0))[0])
    if rt in [4, 5, 6]:
        rPVC_sub['Pos'] -= rt+2 
        x_max = CP_DSPs[1][rt].shape[0]-0.5
        x_min = -rt-2-0.5
    else:
        x_max = CP_DSPs[1][rt].shape[0]-0.5
        x_min = -0.5
    
    fig = plt.figure(figsize=(4, 2))
    ax = Clear_Axes(plt.axes(), close_spines=['right', 'top'], ifxticks=True,ifyticks=True)
    sns.lineplot(
        x="Pos",
        y="aPVC",
        hue="Route",
        data=apvc_sub,
        hue_order=[0, rt],
        ax=ax,
        linewidth=0.5,
        err_kws={"edgecolor": None}
    )
    sns.lineplot(
        x="Pos",
        y="rPVC",
        hue="Type",
        data=rPVC_sub,
        palette=[real[1], ctrl[1]],
        hue_order=['Real', 'Ctrl'],
        ax=ax,
        linewidth=0.5,
        err_kws={"edgecolor": None}
    )
    ax.set_xlim(x_min, 25)
    plt.show()


  0%|          | 0/7 [00:00<?, ?it/s]

d:\Software\Anaconda3\envs\maze\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
d:\Software\Anaconda3\envs\maze\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
d:\Software\Anaconda3\envs\maze\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: divide by zero encountered in divide
  c /= stddev[:, None]
d:\Software\Anaconda3\envs\maze\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: divide by zero encountered in divide
  c /= stddev[None, :]
d:\Software\Anaconda3\envs\maze\lib\site-packages\mylib\Intervaces_Dsp.py:239: RuntimeWarning: Mean of empty slice
  segment_pvc.append(np.nanmean(pvc, axis=1))
100%|██████████| 7/7 [11:57<00:00, 102.43s/it]


24864


  0%|          | 0/7 [00:01<?, ?it/s]


TypeError: unsupported operand type(s) for -: 'list' and 'int'